# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096969


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:39,  1.42s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:41,  1.59s/it]

Rendering models:  20%|██        | 6/30 [00:06<00:27,  1.13s/it]

Rendering models:  30%|███       | 9/30 [00:07<00:16,  1.24it/s]

...

Rendering models:  63%|██████▎   | 19/30 [00:07<00:03,  3.32it/s]

Rendering models:  73%|███████▎  | 22/30 [00:07<00:01,  4.41it/s]

Rendering models:  83%|████████▎ | 25/30 [00:07<00:00,  5.79it/s]

Rendering models:  97%|█████████▋| 29/30 [00:07<00:00,  7.57it/s]

not-logged-in-3b0b929f4475fd276561    0.001830
not-logged-in-3b0b929f4475fd276561    0.001060
not-logged-in-3b0b929f4475fd276561    0.000281
aliabdulhameed                        0.000712
Khai_Noct                             0.000480
not-logged-in-8cf2e6929490167ca6ab    0.008249
jnarayanbvg                           0.008210
gracie_solveig                        0.000901
jaksonA                               0.001362
miyosolange                           0.001361
not-logged-in-22877e3a7af4aafb750a    0.000630
not-logged-in-ae4aeb5d055d833e37bc    0.001578
crush202020                           0.000263
Maverna                               0.003651
AlexYoung35                           0.000746
acapirala                             0.000240
not-logged-in-4c6d97421fdb5b2fcda3    0.000778
Lavadude                              0.022558
jordanrushworth                       0.001627
2022Logan                             0.000323
OneMorePerson                         0.000224
astronautlady

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())